In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.linear_model import LinearRegression
import time
import datetime

In [2]:
tripdata = pd.read_csv("../../dataset/final_bike_sharing.csv")

In [3]:
tripdata.head(3)

,index,tripduration,start_station_id,start_station_name,start_lat,start_lon,end_station_id,end_station_name,end_lat,end_lon,...,conditions,date,dist,birthyear,years_old,holiday,day,month,seasons,gender
0,259617,1.02,others,Manila & 1st,40.721651,-74.042884,others,Manila & 1st,40.721651,-74.042884,...,Clear,2018-09-05,1.783012e-12,1994,24,working_day,weekday,September,autumn,male
1,283363,1.02,others,Heights Elevator,40.748716,-74.040443,others,Heights Elevator,40.748716,-74.040443,...,cloudy_rain,2018-10-06,1.782506e-12,1994,24,working_day,weekend,October,autumn,male
2,169168,1.02,others,Van Vorst Park,40.718489,-74.047727,es_3185,City Hall,40.717732,-74.043845,...,Clear,2018-07-09,3.384331e-01,1963,55,working_day,weekday,July,summer,female


In [4]:
tripdata.columns

Index(['index', 'tripduration', 'start_station_id', 'start_station_name',
       'start_lat', 'start_lon', 'end_station_id', 'end_station_name',
       'end_lat', 'end_lon', 'bikeid', 'usertype', 'hour', 'min', 'temp',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'snow',
       'snowdepth', 'windspeed', 'winddir', 'sealevelpressure', 'cloudcover',
       'visibility', 'solarradiation', 'uvindex', 'conditions', 'date', 'dist',
       'birthyear', 'years_old', 'holiday', 'day', 'month', 'seasons',
       'gender'],
      dtype='object')

## Selecting variables according to P values

In [5]:
selected_features = ['tripduration','start_lat','start_lon','end_lat','end_lon','usertype','hour',
                     'temp','feelslike','dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation',
                     'conditions','dist','birthyear','holiday','day','month','gender']

In [6]:
df = tripdata[selected_features]

In [7]:
def split_dataset(df):
    X = df.drop('tripduration', axis=1)
    y = df['tripduration']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print('Train', X_train.shape, y_train.shape)
    print('Test', X_test.shape, y_test.shape)
    return X_train, X_test, y_train, y_test

### AMputation with 5% NA values

In [8]:
## get the sample size such as 5%, 10%,20% etc
def getSampleSize(df, perc=1):
    return round(perc/100 * df.shape[0])

def getAnIndex(index):
    li = []
    for i in index:
        li.append(i)
    return li   

def induceMissingValues(X_train,col_name, perc):
    random.seed(100)
    index = getAnIndex(X_train.index)
    sample_size = getSampleSize(X_train, perc)
    print("sample size is =", sample_size)
    selected_index = random.sample(index, sample_size)
    for i in selected_index:
        X_train.loc[i, col_name]= np.NaN
    return X_train

###====================####
def convert_to_dataframe(y_train):
    return pd.DataFrame(y_train)

def convert_to_series(y_train):
    return y_train.squeeze()

def induceMissingValuesForDrop(X_train,y_train,col_name, perc):
    random.seed(100)
    index = getAnIndex(X_train.index)
    sample_size = getSampleSize(X_train, perc)
    print("sample size is =", sample_size)
    selected_index = random.sample(index, sample_size)
    y_train = convert_to_dataframe(y_train)
    for i in selected_index:
        y_train.loc[i] = np.NaN
        X_train.loc[i,col_name]= np.NaN
    y_train = convert_to_series(y_train)
    return X_train, y_train

In [9]:
def sample_imputation_X(var,X_train,X_test):
    # extract a random sample
    random_sample_train = X_train[var].dropna().sample(X_train[var].isnull().sum(), random_state=0)
    random_sample_test = X_test[var].dropna().sample(X_test[var].isnull().sum(), random_state=0)
     # re-index the randomly extracted sample
    random_sample_train.index = X_train[X_train[var].isnull()].index
    random_sample_test.index = X_test[X_test[var].isnull()].index

    # replace the NA
    X_train.loc[X_train[var].isnull(), var] = random_sample_train
    X_test.loc[X_test[var].isnull(), var] = random_sample_test
    return X_train, X_test

In [10]:
fill_dict ={'start_lat': 40.79865054, 'start_lon': -74.2404433, 'end_lat': 40.79165072 , 'end_lon':-74.28593088,'usertype':'agent',
            'hour': 24, 'temp': 38.6, 'feelslike':49.4,'dew':-25.7,'snowdepth':5.10,'winddir':380.0,'sealevelpressure':819.3,
            'visibility':7.2,'solarradiation': 1109,'conditions': 'muddy','dist': 5.345,'birthyear': 2003,
            'holiday':'unknown','day': 'special','month': 'unknown','gender': 'nonbinary'
        }

In [11]:
def ampute_each_variables(variable, method, X_train, y_train, X_test, y_test):
    if method == "remove":
        X_train , y_train = induceMissingValuesForDrop(X_train,y_train,variable,perc=20)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("y_train Amputed value count of varaible ", y_train.isnull().sum())
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test , y_test = induceMissingValuesForDrop(X_test,y_test,variable,perc=20)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("y_test Amputed value count of varaible ", y_test.isnull().sum())
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    else:
        X_train  = induceMissingValues(X_train,variable,perc=20)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test  = induceMissingValues(X_test,variable,perc=20)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    
    if method == "remove":
        print("Simulate Imputation for NA")
        X_train.dropna(axis=0, inplace=True)
        y_train.dropna(inplace=True)
        print('Train after NA value removal Imputation', X_train.shape, y_train.shape)
        X_test.dropna(axis=0, inplace=True)
        y_test.dropna(inplace=True)
        print('Test after NA value removal Imputation', X_test.shape, y_test.shape)
    if method == "mode":
        print("Simulate Imputation for Mode")
        X_train.fillna(X_train[variable].mode()[0], inplace=True)
        print('Train after Mode Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test[variable].mode()[0], inplace=True)
        print('Test after Mode Imputation', X_test.shape, y_test.shape)
    if method == "mean":
        print("Simulate Imputation for Mean")
        X_train.fillna(X_train[variable].mean(), inplace=True)
        print('Train after Mean Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test[variable].mean(), inplace=True)
        print('Test after Mean Imputation', X_test.shape, y_test.shape)
    if method == "sample":
        print("Simple Imputation using sampling")
        X_train, X_test = sample_imputation_X(variable, X_train, X_test)
    if method == "bad":
        print("Simulate Imputation for BAD")
        X_train.fillna(fill_dict[variable], inplace=True)
        print('Train  after imputation', X_train.shape, y_train.shape)
        X_test.fillna(fill_dict[variable], inplace=True)
        print('Test  after imputation', X_test.shape, y_test.shape)
        
    return X_train , y_train, X_test , y_test

In [12]:
def one_hot_encoding(X_train, X_test):
    categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = X_train.select_dtypes(include=['int', 'float']).columns.tolist()
    ohe = OneHotEncoder(drop='first', sparse=False)
    X_train_encoded = ohe.fit_transform(X_train[categorical_features])
    X_test_encoded = ohe.fit_transform(X_test[categorical_features])
    X_train_new = np.hstack((X_train[numerical_cols].values, X_train_encoded))
    X_test_new = np.hstack((X_test[numerical_cols].values, X_test_encoded))
    return X_train_new, X_test_new

In [13]:
def model_evaluation(X_train,y_train):
    print(datetime.datetime.now())
    model_lr = LinearRegression(fit_intercept=True).fit(X_train, y_train) 
    print(datetime.datetime.now())
    return model_lr

In [14]:
features_for_amputation = ['start_lat','start_lon','end_lat','end_lon','usertype','hour',
                     'temp','feelslike','dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation',
                     'conditions','dist','birthyear','holiday','day','month','gender']

## 1. Drop NA values 

In [14]:
bootstrap_data_drop ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"remove",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())   
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_drop[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  50469
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  12617
X_test Amputed value count of varaible  12617
Simulate Imputation for NA
Train after NA value removal Imputation (201877, 21) (201877,)
Test after NA value removal Imputation (50470, 21) (50470,)
started bootstrap run is  2023-03-19 21:15:50.346307
stopped bootstrap run is  2023-03-19 21:17:26.763931
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  50469
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (6308

Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  50469
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  12617
X_test Amputed value count of varaible  12617
Simulate Imputation for NA
Train after NA value removal Imputation (201877, 21) (201877,)
Test after NA value removal Imputation (50470, 21) (50470,)
started bootstrap run is  2023-03-19 21:34:35.004867
stopped bootstrap run is  2023-03-19 21:35:53.313592
Now running model is .... solarradiation
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  50469
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  12617
X_test Amputed value count of varaible  12617
Simulate Impu

In [15]:
bootstrap_data_drop

{'start_lat': {'R2': [0.5203954639036867, 0.5380731958239489],
  'RMSE': [2.2047012025161945, 2.254781018554521],
  'intercept': [-691.6287688655016, -343.7083545998907]},
 'start_lon': {'R2': [0.5203954639036867, 0.5380731958239489],
  'RMSE': [2.2047012025161945, 2.254781018554521],
  'intercept': [-691.6287688655016, -343.7083545998907]},
 'end_lat': {'R2': [0.5203954639036867, 0.5380731958239489],
  'RMSE': [2.2047012025161945, 2.254781018554521],
  'intercept': [-691.6287688655016, -343.7083545998907]},
 'end_lon': {'R2': [0.5203954639036867, 0.5380731958239489],
  'RMSE': [2.2047012025161945, 2.254781018554521],
  'intercept': [-691.6287688655016, -343.7083545998907]},
 'usertype': {'R2': [0.5203954639036867, 0.5380731958239489],
  'RMSE': [2.2047012025161945, 2.254781018554521],
  'intercept': [-691.6287688655016, -343.7083545998907]},
 'hour': {'R2': [0.5203954639036867, 0.5380731958239489],
  'RMSE': [2.2047012025161945, 2.254781018554521],
  'intercept': [-691.6287688655016, 

### Mode Replacement

In [16]:
bootstrap_data_mode ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mode",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())  
    
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_mode[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-19 21:48:37.488561
stopped bootstrap run is  2023-03-19 21:50:12.188419
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (6308

Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-19 22:11:31.698887
stopped bootstrap run is  2023-03-19 22:13:00.047901
Now running model is .... dist
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-19 22:13:07.476713
stopped bootstrap run is  2023-03-19 22:14:50.868843
Now running model is .... birthyear
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test 

In [17]:
bootstrap_data_mode

{'start_lat': {'R2': [0.519677496344361, 0.5358137787719836],
  'RMSE': [2.208365700340666, 2.2557877544066596],
  'intercept': [-659.9516015150037, -356.5396646163889]},
 'start_lon': {'R2': [0.5192495534874192, 0.5353936822569065],
  'RMSE': [2.208895477803175, 2.2567168551933188],
  'intercept': [-613.1026755478363, -303.6039582613678]},
 'end_lat': {'R2': [0.5197296025432198, 0.5357804325534078],
  'RMSE': [2.2083316802864035, 2.25582465527527],
  'intercept': [-689.54877950303, -390.35338553154094]},
 'end_lon': {'R2': [0.519637882241262, 0.5357009821676889],
  'RMSE': [2.208694210127952, 2.256286613320034],
  'intercept': [-767.5820869415741, -450.6937301565103]},
 'usertype': {'R2': [0.516792053569199, 0.5331499579920376],
  'RMSE': [2.2159538416493554, 2.262758097134717],
  'intercept': [-653.9457686802841, -370.14698000943883]},
 'hour': {'R2': [0.5190601882516519, 0.5350950490012549],
  'RMSE': [2.209809459035077, 2.2568625694069],
  'intercept': [-654.9999704269615, -369.740

### Mean Replacement

In [18]:
variables_for_mean_amputation = ['start_lat','start_lon','end_lat','end_lon','hour',
                                 'temp','feelslike','snowdepth','winddir','visibility','sealevelpressure',
                                 'solarradiation','dist','birthyear']

bootstrap_data_mean = {}
for var in variables_for_mean_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mean",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())  
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_mean[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simulate Imputation for Mean
Train after Mean Imputation (252346, 21) (252346,)
Test after Mean Imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-19 22:22:55.043319
stopped bootstrap run is  2023-03-19 22:24:22.059354
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simulate Imputation for Mean
Train after Mean Imputation (252346, 21) (252346,)
Test after Mean Imputation (6308

In [19]:
bootstrap_data_mean

{'start_lat': {'R2': [0.5196738417261666, 0.535813724384131],
  'RMSE': [2.2083580618283367, 2.2558109478602324],
  'intercept': [-665.5101996647376, -357.10213916156204]},
 'start_lon': {'R2': [0.5192744432585058, 0.5354095603433926],
  'RMSE': [2.208835568400292, 2.25659702417338],
  'intercept': [-607.2968801785521, -287.0715924160652]},
 'end_lat': {'R2': [0.5197248688840084, 0.5357811978503365],
  'RMSE': [2.208320126821838, 2.2558064391503594],
  'intercept': [-696.7678424818498, -395.3291471071947]},
 'end_lon': {'R2': [0.5196347308271758, 0.5357051528461354],
  'RMSE': [2.2086641840168166, 2.256308659769593],
  'intercept': [-773.8083187070775, -459.76859632420036]},
 'hour': {'R2': [0.5194331363119837, 0.5354799171168252],
  'RMSE': [2.2090060126998474, 2.2561626412014535],
  'intercept': [-649.5101459987345, -367.78146672305434]},
 'temp': {'R2': [0.5196636404415433, 0.5357082079147776],
  'RMSE': [2.2084635751629667, 2.255846557170239],
  'intercept': [-644.1034112935585, -3

### Sampling Replacement

In [20]:
bootstrap_data_sample ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"sample",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())  
    
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_sample[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simple Imputation using sampling
started bootstrap run is  2023-03-19 22:45:47.538154
stopped bootstrap run is  2023-03-19 22:47:17.226335
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simple Imputation using sampling
started bootstrap run is  2023-03-19 22:47:24.025829
stopped bootstrap run is  2023-03-19 22:48:51.894930
Now running model is .... end_lat
Train (252346, 21) (252346,)
Test 

started bootstrap run is  2023-03-19 23:18:35.132656
stopped bootstrap run is  2023-03-19 23:20:27.230881
Now running model is .... day
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simple Imputation using sampling
started bootstrap run is  2023-03-19 23:20:35.376032
stopped bootstrap run is  2023-03-19 23:22:27.382701
Now running model is .... month
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simple Imputation using sampling
started bootstrap run is  2023-03-19 23:22:36.390111
stopped bootstrap run is 

In [21]:
bootstrap_data_sample

{'start_lat': {'R2': [0.5196778163578526, 0.5358073481202144],
  'RMSE': [2.2083697396129622, 2.255800587075448],
  'intercept': [-648.8487075219036, -354.0158429711802]},
 'start_lon': {'R2': [0.519030455538321, 0.5352486934567937],
  'RMSE': [2.2094650249587025, 2.257262926649428],
  'intercept': [-720.6619075113608, -423.29681507574816]},
 'end_lat': {'R2': [0.5196829199610159, 0.5357330208287833],
  'RMSE': [2.208539479746804, 2.2559714268436277],
  'intercept': [-656.4578568032588, -354.0668728324503]},
 'end_lon': {'R2': [0.5194883516628046, 0.5354655697801434],
  'RMSE': [2.2091056941053018, 2.256670872942986],
  'intercept': [-673.6125191923256, -378.40183279547875]},
 'usertype': {'R2': [0.5145210786663359, 0.5310718118136029],
  'RMSE': [2.22147784042969, 2.2677838343582026],
  'intercept': [-659.0124443576717, -375.0220375793198]},
 'hour': {'R2': [0.5187592324047668, 0.5347620577756291],
  'RMSE': [2.210359331683559, 2.257976159851853],
  'intercept': [-651.1484455431826, -

### Bad Imputation

In [15]:
bootstrap_data_bad ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"bad",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())  
    
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_bad[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simulate Imputation for BAD
Train  after imputation (252346, 21) (252346,)
Test  after imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-21 14:49:04.540769
stopped bootstrap run is  2023-03-21 14:51:46.140298
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simulate Imputation for BAD
Train  after imputation (252346, 21) (252346,)
Test  after imputation (63087, 21) (63087,)
st

stopped bootstrap run is  2023-03-21 15:32:11.284370
Now running model is .... dist
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simulate Imputation for BAD
Train  after imputation (252346, 21) (252346,)
Test  after imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-21 15:32:26.911048
stopped bootstrap run is  2023-03-21 15:34:58.378483
Now running model is .... birthyear
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simulate Imputation for BAD
Train  after imputation (252346, 21) (252346,

In [16]:
bootstrap_data_bad

{'start_lat': {'R2': [0.5196077174885214, 0.5357394641989492],
  'RMSE': [2.2084705103941387, 2.255890121933666],
  'intercept': [-583.5878144503172, -300.4959695746703]},
 'start_lon': {'R2': [0.5186983841955478, 0.5349234725874275],
  'RMSE': [2.2105754033895852, 2.2582658309501684],
  'intercept': [-931.1200848090441, -648.499913816966]},
 'end_lat': {'R2': [0.5195260554812042, 0.5356095502769578],
  'RMSE': [2.208756688220107, 2.2561884182461696],
  'intercept': [-537.675792762844, -255.14192678014382]},
 'end_lon': {'R2': [0.5187726262251922, 0.5349280456321894],
  'RMSE': [2.2102552161122198, 2.257851980224068],
  'intercept': [-422.5215491357882, -146.12952313786474]},
 'usertype': {'R2': [0.5168485665893423, 0.5331844673567382],
  'RMSE': [2.215719194371389, 2.2624837020366337],
  'intercept': [-653.164779774355, -371.29581847848226]},
 'hour': {'R2': [0.5184632247856004, 0.5344341613097339],
  'RMSE': [2.2112640568327233, 2.258722942804421],
  'intercept': [-656.9686721196895,